
# Equipo 34 — Fase 1 (Pipeline ML reproducible con DVC)

**Rol:** ML Engineer — *Emilio Contreras*  
**Dataset:** South German Credit (versión limpia `german_credit_cleaned_v1.csv`)  
**Pipeline:** EDA → Preprocess → Train → Evaluate (scikit-learn) + **DVC**

> Este notebook sirve como *orquestador demostrativo*: ejecuta (o muestra) las etapas clave y reúne evidencias para el PDF y el video.



## ML Canvas — Credit Scoring (South German Credit)

**Predicción objetivo**  
Clasificar solicitantes como *good* / *bad* credit risk.

**Usuarios / Stakeholders**  
Analistas de riesgo, áreas de crédito, cumplimiento/regulación, clientes finales.

**Decisiones**  
Aprobación/rechazo; condiciones del préstamo (tasa, monto, garantías).

**Fuentes de datos**  
South German Credit (1,000 filas, 21 columnas). Variables demográficas, historial bancario, condiciones del préstamo.

**Métricas de éxito**  
AUC-ROC (principal), F1 (clase minoritaria), Recall (malos pagadores), matriz de confusión por grupo.

**Restricciones / Riesgos**  
Desbalanceo de clases; sesgo/ética (edad, género, estado civil); explicabilidad requerida.

**Pipeline (alto nivel)**  
EDA → Limpieza/Imputación → Codificación/Scaling → Split estratificado → Modelos (LogReg, RF, XGB) → Tuning → Interpretabilidad (SHAP) → Reporte.

**Valor**  
Reduce pérdidas por morosidad y mejora inclusión financiera con criterios consistentes y auditables.



## Requisitos y estructura sugerida

```bash
pip install -U pandas numpy scikit-learn matplotlib seaborn dvc pyyaml joblib shap
```

```
.
├─ data/
│  ├─ raw/
│  │  └─ german_credit_cleaned_v1.csv
│  └─ interim/
├─ models/
├─ reports/
│  ├─ figures/
│  └─ metrics.json
├─ src/
│  ├─ utils.py
│  ├─ eda.py
│  ├─ preprocess.py
│  ├─ train.py
│  └─ evaluate.py
├─ params.yaml
└─ dvc.yaml
```


In [ ]:

# Parámetros clave del proyecto (ajusta si tu target tiene otro nombre)
DATA_RAW = "data/raw/german_credit_cleaned_v1.csv"
INTERIM = "data/interim/clean.parquet"
MODEL_PATH = "models/model.pkl"
REPORT_PATH = "reports/metrics.json"
TARGET = "credit_risk"  # ajusta si no coincide con tu CSV
RANDOM_STATE = 42


## EDA (resumen rápido en notebook)

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

df = pd.read_csv(DATA_RAW)
display(df.head())
display(df.describe(include='all').transpose().head(20))

Path("reports/figures").mkdir(parents=True, exist_ok=True)

num_cols = df.select_dtypes(include='number').columns.tolist()[:6]
for c in num_cols:
    ax = df[c].hist(bins=30)
    ax.set_title(f"Histograma - {c}")
    fig = ax.get_figure()
    fig.savefig(f"reports/figures/hist_{c}.png", bbox_inches="tight")
    plt.close(fig)

print("Figuras guardadas en reports/figures/")


## Preprocesamiento (imputación, OHE, scaling)

In [ ]:

import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

if TARGET not in df.columns:
    raise ValueError(f"Target '{TARGET}' no está en columnas: {list(df.columns)}")

y = df[TARGET]
X = df.drop(columns=[TARGET])

num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ]
)

X_proc = pre.fit_transform(X)

Path("data/interim").mkdir(parents=True, exist_ok=True)
np.save(INTERIM.replace(".parquet", "_X.npy"), X_proc)
y.to_frame(name=TARGET).to_parquet(INTERIM.replace(".parquet", "_y.parquet"))
joblib.dump(pre, "models/preprocess.joblib")

X_proc.shape, y.shape


## Entrenamiento y validación cruzada

In [ ]:

import numpy as np, pandas as pd, joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

y = pd.read_parquet(INTERIM.replace(".parquet", "_y.parquet")).iloc[:,0]
X = np.load(INTERIM.replace(".parquet", "_X.npy"))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
model = RandomForestClassifier(n_estimators=300, max_depth=12, random_state=RANDOM_STATE)

scores = cross_val_score(model, X, y, cv=cv, scoring="f1_macro")
print(f"CV f1_macro: {scores.mean():.4f} (+/- {scores.std():.4f})")

model.fit(X, y)
Path("models").mkdir(parents=True, exist_ok=True)
joblib.dump(model, MODEL_PATH)
MODEL_PATH


## Evaluación (accuracy, F1 macro, ROC-AUC si aplica)

In [ ]:

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import json

model = joblib.load(MODEL_PATH)
y_pred = model.predict(X)

metrics = {
    "accuracy": float(accuracy_score(y, y_pred)),
    "f1_macro": float(f1_score(y, y_pred, average="macro")),
}

# AUC si el modelo soporta predict_proba
try:
    y_prob = model.predict_proba(X)[:,1]
    metrics["roc_auc"] = float(roc_auc_score(y, y_prob))
except Exception:
    pass

cm = confusion_matrix(y, y_pred).tolist()
metrics["confusion_matrix"] = cm

Path("reports").mkdir(parents=True, exist_ok=True)
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)

metrics



## Versionado de datos con DVC (guía rápida)

```bash
git init
dvc init

# Rastrear dataset "raw"
dvc add data/raw/german_credit_cleaned_v1.csv
git add data/raw/*.dvc .gitignore
git commit -m "Track raw cleaned dataset with DVC"

# Ejecutar pipeline (si usas scripts y dvc.yaml)
dvc repro

# Ver métricas
dvc metrics show
```



## Siguientes pasos / Evidencias para PDF y Video
- Insertar capturas de `reports/figures/*` (histogramas, etc.).  
- Pegar la tabla de métricas generada (`reports/metrics.json`).  
- Añadir interpretabilidad (SHAP) si se requiere explicabilidad adicional.  
- Documentar roles, commits y liga al video dentro del PDF final.
